In [ ]:
# -*- coding: utf-8 -*-

import csv
import json
import sys

In [ ]:
# Files
LEXICON_FILE = 'NRC-Emotion-Lexicon-v0.92-English.csv'
CATEGORIES_FILE = 'categories.json'

TEXT_FILE = 'review_normal.json'
DATA_FILE = 'output_data.csv'

In [1]:
# Init
vocabulary = []
words = []
categories = {}
category_headers = []
text = {}
data = []
chapters = []

In [2]:
# Read vocabulary
with open(LEXICON_FILE, 'r') as f:
    rows = csv.reader(f, delimiter=',')
    headers = next(rows, None) # remove header
    for row in rows:
        entry = {}
        for i, h in enumerate(headers):
            entry[h] = row[i]
        vocabulary.append(entry)
    #print (vocabulary)
    words = [v['ï»¿english word'] for v in vocabulary]

In [3]:
# Read categories
with open(CATEGORIES_FILE) as f:
    categories = json.load(f)
    emotions = categories['emotion']

In [4]:
# lemmatized filtered etc words from all reviews 
# list of lists
reviews = []
# Read normalized text
with open(TEXT_FILE) as f:
    text = json.load(f)
    for i in range(len(text)):
        reviews.append(text[i])

In [5]:
# Check if word matches any lexicons
def addData(word, review):
    global data
    global vocabulary
    global emotions
    global words

    match = -1
    for i,w in enumerate(words):
        if w==word:
            match = i
            break

    if match >= 0:
        for item in vocabulary:
            if item['ï»¿english word'] == word:
                entry = item
                break
            else: 
                continue
        row = [None]*10
        i = 0
        row[i] = entry['\xef\xbb\xbfenglish word']
        for emotion in emotions:
            i += 1
            if entry[emotion]:
                row[i] = entry[emotion]
            else:
                row[i] = -1
        row[i+1] = review
        data.append(row)

In [6]:
# Read each review
data = []
for i, review in enumerate(reviews):
  #  wordss = review.split(' ')
    for word in review:
        addData(word, i)

In [7]:
import os
if os.path.isfile('output_data.csv'):
    os.remove('output_data.csv')

In [8]:
# Output data as csv
with open(DATA_FILE, 'w') as f:
    cw = csv.writer(f)
    headers = ['english word']
    for emotion in emotions:
        headers.append(emotion)
    headers.append(u'review')
    cw.writerow(headers)
    for row in data:
        cw.writerow(row)
    print('Successfully wrote '+str(len(data))+' entries to file: '+DATA_FILE)

Successfully wrote 205 entries to file: output_data.csv
